In [231]:
import pandas as pd

def translate_team(team_name):
    # Check if translation exists, otherwise return the original name
    return translation_dict.get(team_name, f"Unknown ({team_name})")

Head_path = 'D:\Projects\Other_exp\Jupyter\orie4741-fp\prediction_data/'
# Step 1: Load the prediction CSV
prediction_df = pd.read_csv(Head_path+'prediction.csv')

# Step 2: Load the translation CSV (assuming it has 'Chinese' and 'English' columns)
translation_df = pd.read_csv(Head_path+'translation.csv', encoding='utf-8')

# Step 3: Create a dictionary for mapping Chinese team names to English
translation_dict = pd.Series(translation_df['English'].values, index=translation_df['Chinese'].values).to_dict()

# Step 3: Create a dictionary for mapping Chinese aHh to Math
translation_dict_math = pd.Series(translation_df['Math_Hh'].values, index=translation_df['Chinese_Hh'].values).to_dict()

# Step 4: Replace Chinese team names with English in HomeTeam and AwayTeam columns
prediction_df['HomeTeam'] = prediction_df['HomeTeam'].map(translation_dict).fillna(prediction_df['HomeTeam'])
prediction_df['AwayTeam'] = prediction_df['AwayTeam'].map(translation_dict).fillna(prediction_df['AwayTeam'])
prediction_df['AHh'] = prediction_df['AHh'].map(translation_dict_math).fillna(prediction_df['AHh'])
# prediction_df['HomeTeam'] = prediction_df['HomeTeam'].apply(translate_team)
# prediction_df['AwayTeam'] = prediction_df['AwayTeam'].apply(translate_team)

# Step 5: Save the updated dataframe (optional)
prediction_df.to_csv(Head_path+'updated_prediction.csv', index=False, encoding='utf-8-sig')
print(prediction_df)

    Unnamed: 0  Div       Date     HomeTeam       AwayTeam  B365H  B365D  \
0            1   I1  2025/1/25         Como       Atalanta   4.20   3.60   
1            2   D1  2025/1/25     Augsburg     Heidenheim   1.85   3.60   
2            3   D1  2025/1/25     Dortmund  Werder Bremen   1.62   4.33   
3            4   D1  2025/1/25     Freiburg  Bayern Munich   7.50   5.75   
4            5   D1  2025/1/25   RB Leipzig     Leverkusen   3.60   3.80   
5            6   E0  2025/1/25       Wolves        Arsenal   6.50   4.50   
6            7   E0  2025/1/25    Liverpool        Ipswich   1.11  12.00   
7            8   E0  2025/1/25  Southampton      Newcastle   7.00   5.25   
8            9   E0  2025/1/25     Brighton        Everton   1.67   3.75   
9           10   E0  2025/1/25  Bournemouth  Nott'm Forest   2.05   3.70   
10          11  SP1  2025/1/25   Ath Madrid     Villarreal   1.67   4.00   
11          12   I1  2025/1/25       Napoli       Juventus   2.15   3.10   
12          

In [232]:
# import os
# 
# # 文件夹路径
# folder_path = 'D:\Projects\Other_exp\Jupyter\orie4741-fp\elo/'
# 
# # 获取文件夹下所有文件的名称
# file_names = []
# for root, dirs, files in os.walk(folder_path):
#     for file in files:
#         # 分离文件名和扩展名，获取文件名部分
#         file_name_without_extension = os.path.splitext(file)[0]
#         file_names.append(file_name_without_extension)
# 
# # 将文件名称写入到 txt 文件，按行存储
# with open(folder_path + 'file_names.txt', 'w', encoding='utf-8') as f:
#     for file_name in file_names:
#         f.write(file_name + '\n')
# 
# print("文件名称已成功保存到 file_names.txt")

In [233]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import joblib
from joblib import load, dump
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [234]:
df = pd.read_csv('prediction_data/updated_prediction.csv', low_memory = False)
df

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,AHh,B365AHH,...,MLP_new,Unnamed: 18,Rand_SVM,Unnamed: 20,SVM,Unnamed: 22,Unnamed: 23,Unnamed: 24,truth_balance,truth
0,1,I1,2025/1/25,Como,Atalanta,4.20,3.60,1.85,0.50,2.02,...,2,2,2,2,2,NaN,NaN,NaN,1,1
1,2,D1,2025/1/25,Augsburg,Heidenheim,1.85,3.60,4.00,-0.50,1.91,...,2,0,0,0,0,NaN,NaN,NaN,-1,1
2,3,D1,2025/1/25,Dortmund,Werder Bremen,1.62,4.33,5.00,-1.00,2.05,...,2,0,0,0,0,NaN,NaN,NaN,-1,0
3,4,D1,2025/1/25,Freiburg,Bayern Munich,7.50,5.75,1.33,1.50,2.00,...,0,2,2,1;0,0,NaN,NaN,NaN,2,2
4,5,D1,2025/1/25,RB Leipzig,Leverkusen,3.60,3.80,1.91,0.50,1.95,...,0,2,2,2,2,NaN,NaN,NaN,1,2
5,6,E0,2025/1/25,Wolves,Arsenal,6.50,4.50,1.45,1.25,1.83,...,2,1;2,2,0,0,NaN,NaN,NaN,1,1
6,7,E0,2025/1/25,Liverpool,Ipswich,1.11,12.00,15.00,-2.50,1.87,...,0,1;0,0,1;0,0,NaN,NaN,NaN,-2,2
7,8,E0,2025/1/25,Southampton,Newcastle,7.00,5.25,1.40,1.25,2.03,...,2,2,2,0,0,NaN,NaN,NaN,1,0
8,9,E0,2025/1/25,Brighton,Everton,1.67,3.75,5.50,-0.75,1.87,...,2,1;2,2,1;2,2,NaN,NaN,NaN,-1,0
9,10,E0,2025/1/25,Bournemouth,Nott'm Forest,2.05,3.70,3.40,-0.25,1.83,...,2,1;2,2,1;2,2,NaN,NaN,NaN,-1,2


In [235]:
conditions = [
    df['AHh'] >= 2.5,         # AHh >= 2.75
    (df['AHh'] >= 1.5) & (df['AHh'] <= 2.25),  # 1.75 < AHh <= 2.75
    (df['AHh'] >= 0.25) & (df['AHh'] <= 1.25),  # 0.25 < AHh <= 1.75
    df['AHh'] == 0,            # AHh == 0
    (df['AHh'] >= -1.25) & (df['AHh'] <= -0.25),  # -1.75 < AHh <= -0.25
    (df['AHh'] >= -2.25) & (df['AHh'] <= -1.5),  # -2.75 < AHh <= -1.75
    df['AHh'] <= -2.5
]
labels = [3, 2, 1, 0, -1, -2, -3]

df['balance_val'] = np.select(conditions, labels)
df

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,AHh,B365AHH,...,Unnamed: 18,Rand_SVM,Unnamed: 20,SVM,Unnamed: 22,Unnamed: 23,Unnamed: 24,truth_balance,truth,balance_val
0,1,I1,2025/1/25,Como,Atalanta,4.20,3.60,1.85,0.50,2.02,...,2,2,2,2,NaN,NaN,NaN,1,1,1
1,2,D1,2025/1/25,Augsburg,Heidenheim,1.85,3.60,4.00,-0.50,1.91,...,0,0,0,0,NaN,NaN,NaN,-1,1,-1
2,3,D1,2025/1/25,Dortmund,Werder Bremen,1.62,4.33,5.00,-1.00,2.05,...,0,0,0,0,NaN,NaN,NaN,-1,0,-1
3,4,D1,2025/1/25,Freiburg,Bayern Munich,7.50,5.75,1.33,1.50,2.00,...,2,2,1;0,0,NaN,NaN,NaN,2,2,2
4,5,D1,2025/1/25,RB Leipzig,Leverkusen,3.60,3.80,1.91,0.50,1.95,...,2,2,2,2,NaN,NaN,NaN,1,2,1
5,6,E0,2025/1/25,Wolves,Arsenal,6.50,4.50,1.45,1.25,1.83,...,1;2,2,0,0,NaN,NaN,NaN,1,1,1
6,7,E0,2025/1/25,Liverpool,Ipswich,1.11,12.00,15.00,-2.50,1.87,...,1;0,0,1;0,0,NaN,NaN,NaN,-2,2,-3
7,8,E0,2025/1/25,Southampton,Newcastle,7.00,5.25,1.40,1.25,2.03,...,2,2,0,0,NaN,NaN,NaN,1,0,1
8,9,E0,2025/1/25,Brighton,Everton,1.67,3.75,5.50,-0.75,1.87,...,1;2,2,1;2,2,NaN,NaN,NaN,-1,0,-1
9,10,E0,2025/1/25,Bournemouth,Nott'm Forest,2.05,3.70,3.40,-0.25,1.83,...,1;2,2,1;2,2,NaN,NaN,NaN,-1,2,-1


In [236]:
# remove unknown columns
columns = df.columns
unknown_cols = [c for c in columns if "Unnamed" in c]
df = df.drop(columns=unknown_cols)

# remove rows with only na values
df = df.dropna(how="all")

features_to_keep = ["Div", "Date", "HomeTeam", "AwayTeam", "B365H", "B365D", "B365A","AHh", "B365AHH", "B365AHA", "balance_val"
                    ]
df = df[features_to_keep]

In [237]:
from os.path import exists
all_teams = set(df["HomeTeam"].unique()).union(set(df["AwayTeam"].unique()))

dfs = {}

for team in all_teams:
    if exists("elo/" + team + ".csv"):
        dfs[team] = pd.read_csv("elo/" + team + ".csv")
def find_team_elo_at(team_name, date):
    elo_df = dfs.get(team_name, None)
    if elo_df is None:
        return None
    elo_df["From"] = pd.to_datetime(elo_df["From"], format='mixed', dayfirst=True)
    elo_df["To"] = pd.to_datetime(elo_df["To"], format='mixed', dayfirst=True)
    if date > '2000-05-01':
        aa = 1
    # if date > '2024-10-01':
    #     aa = 2
    try:
        # elo = elo_df.loc[elo_df["From"] >= date]["Elo"].iloc[0]
        # print(elo_df.loc[(elo_df["From"] <= date) & (elo_df["To"] > date)])
        elo = elo_df.loc[(elo_df["From"] <= date) & (elo_df["To"] >= date)]["Elo"].iloc[0]
        return elo
    except:
        return None
    
def add_elo_feature(s):
    s["HomeTeamELO"] = find_team_elo_at(s["HomeTeam"], s["Date"])
    s["AwayTeamELO"] = find_team_elo_at(s["AwayTeam"], s["Date"])
    return s

f = np.vectorize(find_team_elo_at)

df["HomeTeamELO"] = f(df["HomeTeam"], df["Date"])
df["AwayTeamELO"] = f(df["AwayTeam"], df["Date"])

# df = df.apply(add_elo_feature, axis=1)
df.to_csv("prediction_data/updated_prediction_with_elo.csv")
df

,Div,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,AHh,B365AHH,B365AHA,balance_val,HomeTeamELO,AwayTeamELO
0,I1,2025/1/25,Como,Atalanta,4.20,3.60,1.85,0.50,2.02,1.88,1,1609.479370,1906.518799
1,D1,2025/1/25,Augsburg,Heidenheim,1.85,3.60,4.00,-0.50,1.91,1.99,-1,1602.454956,1553.711548
2,D1,2025/1/25,Dortmund,Werder Bremen,1.62,4.33,5.00,-1.00,2.05,1.85,-1,1764.334595,1644.601685
3,D1,2025/1/25,Freiburg,Bayern Munich,7.50,5.75,1.33,1.50,2.00,1.90,2,1651.050293,1887.472534
4,D1,2025/1/25,RB Leipzig,Leverkusen,3.60,3.80,1.91,0.50,1.95,1.95,1,1746.709106,1903.679688
5,E0,2025/1/25,Wolves,Arsenal,6.50,4.50,1.45,1.25,1.83,2.07,1,1663.880493,1981.875488
6,E0,2025/1/25,Liverpool,Ipswich,1.11,12.00,15.00,-2.50,1.87,2.03,-3,2003.649048,1596.177856
7,E0,2025/1/25,Southampton,Newcastle,7.00,5.25,1.40,1.25,2.03,1.87,1,1549.954956,1832.969360
8,E0,2025/1/25,Brighton,Everton,1.67,3.75,5.50,-0.75,1.87,2.03,-1,1778.029663,1705.085083
9,E0,2025/1/25,Bournemouth,Nott'm Forest,2.05,3.70,3.40,-0.25,1.83,2.07,-1,1800.629883,1784.509766


In [238]:
df_noNa = df.dropna()
df_noNa.reset_index(inplace=True, drop=True)
df_noNa

,Div,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,AHh,B365AHH,B365AHA,balance_val,HomeTeamELO,AwayTeamELO
0,I1,2025/1/25,Como,Atalanta,4.20,3.60,1.85,0.50,2.02,1.88,1,1609.479370,1906.518799
1,D1,2025/1/25,Augsburg,Heidenheim,1.85,3.60,4.00,-0.50,1.91,1.99,-1,1602.454956,1553.711548
2,D1,2025/1/25,Dortmund,Werder Bremen,1.62,4.33,5.00,-1.00,2.05,1.85,-1,1764.334595,1644.601685
3,D1,2025/1/25,Freiburg,Bayern Munich,7.50,5.75,1.33,1.50,2.00,1.90,2,1651.050293,1887.472534
4,D1,2025/1/25,RB Leipzig,Leverkusen,3.60,3.80,1.91,0.50,1.95,1.95,1,1746.709106,1903.679688
5,E0,2025/1/25,Wolves,Arsenal,6.50,4.50,1.45,1.25,1.83,2.07,1,1663.880493,1981.875488
6,E0,2025/1/25,Liverpool,Ipswich,1.11,12.00,15.00,-2.50,1.87,2.03,-3,2003.649048,1596.177856
7,E0,2025/1/25,Southampton,Newcastle,7.00,5.25,1.40,1.25,2.03,1.87,1,1549.954956,1832.969360
8,E0,2025/1/25,Brighton,Everton,1.67,3.75,5.50,-0.75,1.87,2.03,-1,1778.029663,1705.085083
9,E0,2025/1/25,Bournemouth,Nott'm Forest,2.05,3.70,3.40,-0.25,1.83,2.07,-1,1800.629883,1784.509766


In [239]:
# df_noNa_new = df_noNa.iloc[:3]
df_noNa_new = df_noNa

In [240]:
# 与历史数据合并，计算后续特征
history_df = pd.read_csv(Head_path+'history_data_balance.csv')
history_df

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,WHD,WHA,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,easy_label,balance_val
0,F1,2003/8/2,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,1.66,3.10,5.00,-0.75,2.050,1.850,1702.604858,1611.196045,0,-1
1,F1,2003/8/2,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,2.60,3.10,2.40,0.00,1.925,1.975,1685.016113,1665.625732,0,0
2,D1,2003/8/2,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,1.57,3.50,5.00,-0.75,1.800,2.100,1718.566284,1649.805298,0,-1
3,D1,2003/8/2,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,1.72,3.40,4.00,-0.75,2.025,1.875,1719.916748,1692.120972,0,-1
4,F1,2003/8/2,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,1.66,3.30,4.50,-0.75,1.900,2.000,1697.354004,1539.958130,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37293,I1,2025/1/19,Verona,Lazio,0.0,3.0,A,5.000,3.500,1.750,4.50,3.60,1.75,0.75,1.870,2.030,1620.682861,1810.438232,0,1
37294,D1,2025/1/19,Werder Bremen,Augsburg,0.0,2.0,A,1.850,3.800,4.000,1.85,3.70,4.00,-0.50,1.890,2.010,1667.482422,1594.142822,0,-1
37295,E0,2025/1/20,Chelsea,Wolves,3.0,1.0,H,1.360,5.750,6.500,1.36,5.00,7.00,-1.50,1.980,1.920,1859.114746,1669.096313,1,-2
37296,I1,2025/1/20,Como,Udinese,4.0,1.0,H,2.150,3.400,3.400,2.15,3.25,3.40,-0.25,1.870,2.030,1590.385498,1680.550171,2,-1


In [241]:
# 合并两个 DataFrame，重置索引
df_noNa_comb = pd.concat([history_df, df_noNa_new], ignore_index=True)
df_noNa_comb

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,WHD,WHA,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,easy_label,balance_val
0,F1,2003/8/2,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,1.66,3.1,5.0,-0.75,2.050,1.850,1702.604858,1611.196045,0.0,-1
1,F1,2003/8/2,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,2.60,3.1,2.4,0.00,1.925,1.975,1685.016113,1665.625732,0.0,0
2,D1,2003/8/2,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,1.57,3.5,5.0,-0.75,1.800,2.100,1718.566284,1649.805298,0.0,-1
3,D1,2003/8/2,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,1.72,3.4,4.0,-0.75,2.025,1.875,1719.916748,1692.120972,0.0,-1
4,F1,2003/8/2,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,1.66,3.3,4.5,-0.75,1.900,2.000,1697.354004,1539.958130,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37308,SP1,2025/1/25,Ath Madrid,Villarreal,NaN,NaN,NaN,1.670,4.000,5.000,NaN,NaN,NaN,-0.75,1.890,2.010,1856.466064,1735.033813,NaN,-1
37309,I1,2025/1/25,Napoli,Juventus,NaN,NaN,NaN,2.150,3.100,3.800,NaN,NaN,NaN,-0.25,1.850,2.050,1866.017822,1870.678345,NaN,-1
37310,E0,2025/1/25,Man City,Chelsea,NaN,NaN,NaN,1.950,4.100,3.500,NaN,NaN,NaN,-0.50,2.010,1.890,1935.029175,1861.455933,NaN,-1
37311,SP1,2025/1/25,Valladolid,Real Madrid,NaN,NaN,NaN,10.000,6.000,1.270,NaN,NaN,NaN,1.75,1.900,2.000,1545.639893,1949.952026,NaN,2


In [242]:
# # 加载编码器
# div_encoder = joblib.load('div_encoder.pkl')
# home_encoder = joblib.load('home_encoder.pkl')
# away_encoder = joblib.load('away_encoder.pkl')
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

div_encoder = OneHotEncoder()
home_encoder = OneHotEncoder()
away_encoder = OneHotEncoder()
# 你可以使用加载的编码器进行转换
onehot_div = div_encoder.fit_transform(df_noNa_comb.Div.values.reshape(-1,1)).toarray().astype(int)
onehot_div_df = pd.DataFrame(onehot_div, columns = ["Div "+str(int(i)) for i in range(onehot_div.shape[1])])

onehot_home = home_encoder.fit_transform(df_noNa_comb.HomeTeam.values.reshape(-1,1)).toarray().astype(int)
onehot_home_df = pd.DataFrame(onehot_home, columns = ['HomeTeam ' + str(int(i)) for i in np.arange(onehot_home.shape[1])])

onehot_away = away_encoder.fit_transform(df_noNa_comb.AwayTeam.values.reshape(-1,1)).toarray().astype(int)
onehot_away_df = pd.DataFrame(onehot_away, columns = ['AwayTeam ' + str(int(i)) for i in np.arange(onehot_away.shape[1])])

df_noNa_div = pd.concat([df_noNa_comb, onehot_div_df, onehot_home_df, onehot_away_df], axis = 1)
df_noNa_div.drop(columns = ['Div'], inplace = True)
df_noNa_div

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,...,AwayTeam 197,AwayTeam 198,AwayTeam 199,AwayTeam 200,AwayTeam 201,AwayTeam 202,AwayTeam 203,AwayTeam 204,AwayTeam 205,AwayTeam 206
0,2003/8/2,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,1.66,...,0,0,0,0,0,0,0,0,0,0
1,2003/8/2,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,2.60,...,0,0,0,0,0,0,0,0,0,0
2,2003/8/2,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,1.57,...,0,0,0,0,0,0,0,0,0,0
3,2003/8/2,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,1.72,...,0,0,1,0,0,0,0,0,0,0
4,2003/8/2,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,1.66,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37308,2025/1/25,Ath Madrid,Villarreal,NaN,NaN,NaN,1.670,4.000,5.000,NaN,...,1,0,0,0,0,0,0,0,0,0
37309,2025/1/25,Napoli,Juventus,NaN,NaN,NaN,2.150,3.100,3.800,NaN,...,0,0,0,0,0,0,0,0,0,0
37310,2025/1/25,Man City,Chelsea,NaN,NaN,NaN,1.950,4.100,3.500,NaN,...,0,0,0,0,0,0,0,0,0,0
37311,2025/1/25,Valladolid,Real Madrid,NaN,NaN,NaN,10.000,6.000,1.270,NaN,...,0,0,0,0,0,0,0,0,0,0


In [243]:
target_encoder = LabelEncoder()
df_noNa_div['Result'] = target_encoder.fit_transform(df_noNa_div.easy_label) 

In [244]:
df_noNa_div['Year'] = pd.DatetimeIndex(df_noNa_div.Date).year

df_noNa_div['Month'] = pd.DatetimeIndex(df_noNa_div.Date).month
df_noNa_div['Sin_Month'] = np.sin(2*np.pi*df_noNa_div.Month/12)
df_noNa_div['Cos_Month'] = np.cos(2*np.pi*df_noNa_div.Month/12)

df_noNa_div['DayofYear'] = pd.DatetimeIndex(df_noNa_div.Date).dayofyear
df_noNa_div['Sin_Day'] = np.sin(2*np.pi*df_noNa_div.DayofYear/365)
df_noNa_div['Cos_Day'] = np.cos(2*np.pi*df_noNa_div.DayofYear/365)

# 注意 inplace是在原始frame修改，返回值是Nonetype
# learning_df = learning_df_div.drop(columns = ['Date','Month'], inplace = True)
df_noNa_div_date = df_noNa_div.drop(columns = ['Date','Month'])
df_noNa_div_date

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,WHD,...,AwayTeam 204,AwayTeam 205,AwayTeam 206,Result,Year,Sin_Month,Cos_Month,DayofYear,Sin_Day,Cos_Day
0,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,1.66,3.1,...,0,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551
1,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,2.60,3.1,...,0,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551
2,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,1.57,3.5,...,0,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551
3,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,1.72,3.4,...,0,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551
4,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,1.66,3.3,...,0,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37308,Ath Madrid,Villarreal,NaN,NaN,NaN,1.670,4.000,5.000,NaN,NaN,...,0,0,0,3,2025,0.500000,0.866025,25,0.417194,0.908818
37309,Napoli,Juventus,NaN,NaN,NaN,2.150,3.100,3.800,NaN,NaN,...,0,0,0,3,2025,0.500000,0.866025,25,0.417194,0.908818
37310,Man City,Chelsea,NaN,NaN,NaN,1.950,4.100,3.500,NaN,NaN,...,0,0,0,3,2025,0.500000,0.866025,25,0.417194,0.908818
37311,Valladolid,Real Madrid,NaN,NaN,NaN,10.000,6.000,1.270,NaN,NaN,...,0,0,0,3,2025,0.500000,0.866025,25,0.417194,0.908818


In [245]:
# 定义一个函数来计算两队之间上一场比赛的结果
def compute_last_matches(df):
    
    unique_matchups = list(set((list(zip(df.HomeTeam, df.AwayTeam)))))
    df['Last Match Result'] = np.nan
    for home, away in unique_matchups:
        matchup_df = df[(df.HomeTeam == home) & (df.AwayTeam == away)]
        # 使用 shift(1) 方法将 FTR（全场比赛结果）列中的数据向下移动一行，这样每行的 last_match_result 将对应于这两队之前的一场比赛的结果。fill_value='Na' 确保了数据移动后空出的位置填充为 'Na'。
        # last_match_result = matchup_df.FTR.shift(1, fill_value='Na')
        last_match_result = matchup_df.Result.shift(1, fill_value=3)
        df.loc[matchup_df.index, 'Last Match Result'] = last_match_result
        
    lmr_encoder = LabelEncoder()
    df['Last Match Result'] = lmr_encoder.fit_transform(df['Last Match Result'])
    df.drop(columns = ['easy_label'], inplace = True)
    return df
df_noNa_div_date_lastMatch = compute_last_matches(df_noNa_div_date)
df_noNa_div_date_lastMatch

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,WHD,...,AwayTeam 205,AwayTeam 206,Result,Year,Sin_Month,Cos_Month,DayofYear,Sin_Day,Cos_Day,Last Match Result
0,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,1.66,3.1,...,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3
1,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,2.60,3.1,...,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3
2,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,1.57,3.5,...,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3
3,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,1.72,3.4,...,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3
4,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,1.66,3.3,...,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37308,Ath Madrid,Villarreal,NaN,NaN,NaN,1.670,4.000,5.000,NaN,NaN,...,0,0,3,2025,0.500000,0.866025,25,0.417194,0.908818,1
37309,Napoli,Juventus,NaN,NaN,NaN,2.150,3.100,3.800,NaN,NaN,...,0,0,3,2025,0.500000,0.866025,25,0.417194,0.908818,1
37310,Man City,Chelsea,NaN,NaN,NaN,1.950,4.100,3.500,NaN,NaN,...,0,0,3,2025,0.500000,0.866025,25,0.417194,0.908818,0
37311,Valladolid,Real Madrid,NaN,NaN,NaN,10.000,6.000,1.270,NaN,NaN,...,0,0,3,2025,0.500000,0.866025,25,0.417194,0.908818,0


In [246]:
def compute_winstreak(df):
    
    years = df.Year.unique()
    df_lst = []    
    for year in years:
        
        year_df = df[df.Year == year]
        year_df['HomeWin'] = year_df.Result.replace([0, 1, 2], [0, 0, 1])
        year_df['AwayWin'] = year_df.Result.replace([0, 1, 2], [1, 0, 0])
        year_df['HomeWinStreak'] = None
        year_df['AwayWinStreak'] = None
        
        hometeams = year_df.HomeTeam.unique()
        awayteams = year_df.AwayTeam.unique()
        if year > 2024:
            # 将 AwayWin = 3 当作 0 来处理，保持计算连胜记录
            year_df['HomeWin'] = year_df['HomeWin'].replace(3, 1)
            year_df['AwayWin'] = year_df['AwayWin'].replace(3, 1)
        
        for team in hometeams:
            team_df = year_df[(year_df.HomeTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            team_grouper = (team_df.HomeWin != team_df.HomeWin.shift()).cumsum()
            team_df['HomeWinStreak'] = team_df[['HomeWin']].groupby(team_grouper).cumsum()
            team_df.loc[team_df.HomeWinStreak >0, 'HomeWinStreak'] -= 1
            year_df.loc[team_df.index, 'HomeWinStreak'] = team_df.HomeWinStreak
            
        for team in awayteams:
            # if team == 'Arsenal' and year == 2025:
            #     aa = 1
            team_df = year_df[(year_df.AwayTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            team_grouper = (team_df.AwayWin != team_df.AwayWin.shift()).cumsum()
            team_df['AwayWinStreak'] = team_df[['AwayWin']].groupby(team_grouper).cumsum()
            team_df.loc[team_df.AwayWinStreak >0, 'AwayWinStreak'] -= 1
            year_df.loc[team_df.index, 'AwayWinStreak'] = team_df.AwayWinStreak
            
        df_lst.append(year_df)
        
    return pd.concat(df_lst, axis = 0).drop(columns = ['HomeWin', 'AwayWin'])#,'DayofYear'])
df_noNa_div_date_lastMatch_streak = compute_winstreak(df_noNa_div_date_lastMatch)
df_noNa_div_date_lastMatch_streak

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,WHD,...,Result,Year,Sin_Month,Cos_Month,DayofYear,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak
0,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,1.66,3.1,...,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3,0,0
1,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,2.60,3.1,...,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3,0,0
2,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,1.57,3.5,...,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3,0,0
3,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,1.72,3.4,...,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3,0,0
4,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,1.66,3.3,...,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37308,Ath Madrid,Villarreal,NaN,NaN,NaN,1.670,4.000,5.000,NaN,NaN,...,3,2025,0.500000,0.866025,25,0.417194,0.908818,1,0,0
37309,Napoli,Juventus,NaN,NaN,NaN,2.150,3.100,3.800,NaN,NaN,...,3,2025,0.500000,0.866025,25,0.417194,0.908818,1,0,1
37310,Man City,Chelsea,NaN,NaN,NaN,1.950,4.100,3.500,NaN,NaN,...,3,2025,0.500000,0.866025,25,0.417194,0.908818,0,1,0
37311,Valladolid,Real Madrid,NaN,NaN,NaN,10.000,6.000,1.270,NaN,NaN,...,3,2025,0.500000,0.866025,25,0.417194,0.908818,0,1,0


In [247]:
def compute_winstodate(df):
    
    years = df.Year.unique()
    df_lst = []    
    for year in years:
        
        year_df = df[df.Year == year]
        year_df['HomeWin'] = year_df.Result.replace([0, 1, 2], [0, 0, 1])
        year_df['AwayWin'] = year_df.Result.replace([0, 1, 2], [1, 0, 0])
        year_df['HomeWinsToDate'] = None
        year_df['AwayWinsToDate'] = None
        
        hometeams = year_df.HomeTeam.unique()
        awayteams = year_df.AwayTeam.unique()
        if year > 2024:
            # 将 AwayWin = 3 当作 0 来处理，保持计算连胜记录
            year_df['HomeWin'] = year_df['HomeWin'].replace(3, 0)
            year_df['AwayWin'] = year_df['AwayWin'].replace(3, 0)
        for team in hometeams:
            team_df = year_df[(year_df.HomeTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            team_df['HomeWinsToDate'] = team_df.HomeWin.cumsum()
            year_df.loc[team_df.index, 'HomeWinsToDate'] = team_df.HomeWinsToDate
            
        for team in awayteams:
            if team == 'Arsenal' and year == 2025:
                aa = 1
            team_df = year_df[(year_df.AwayTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))
            
            team_df['AwayWinsToDate'] = team_df.AwayWin.cumsum()
            year_df.loc[team_df.index, 'AwayWinsToDate'] = team_df.AwayWinsToDate
            
        df_lst.append(year_df)
        
    return pd.concat(df_lst, axis = 0).drop(columns = ['HomeWin', 'AwayWin','DayofYear'])
df_noNa_div_date_lastMatch_streak_windate = compute_winstodate(df_noNa_div_date_lastMatch_streak)
df_noNa_div_date_lastMatch_streak_windate.drop(columns = ['HomeTeam', 'AwayTeam'], inplace = True)
df_noNa_div_date_lastMatch_streak_windate

,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,WHD,WHA,AHh,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
0,1.0,2.0,A,1.727,3.100,4.500,1.66,3.1,5.0,-0.75,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
1,0.0,1.0,A,2.500,2.875,2.625,2.60,3.1,2.4,0.00,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
2,0.0,3.0,A,1.571,3.500,5.000,1.57,3.5,5.0,-0.75,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
3,0.0,3.0,A,1.833,3.200,3.750,1.72,3.4,4.0,-0.75,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
4,0.0,0.0,D,1.571,3.250,5.500,1.66,3.3,4.5,-0.75,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37308,NaN,NaN,NaN,1.670,4.000,5.000,NaN,NaN,NaN,-0.75,...,2025,0.500000,0.866025,0.417194,0.908818,1,0,0,0,0
37309,NaN,NaN,NaN,2.150,3.100,3.800,NaN,NaN,NaN,-0.25,...,2025,0.500000,0.866025,0.417194,0.908818,1,0,1,0,1
37310,NaN,NaN,NaN,1.950,4.100,3.500,NaN,NaN,NaN,-0.50,...,2025,0.500000,0.866025,0.417194,0.908818,0,1,0,1,0
37311,NaN,NaN,NaN,10.000,6.000,1.270,NaN,NaN,NaN,1.75,...,2025,0.500000,0.866025,0.417194,0.908818,0,1,0,1,0


In [248]:
betting_feats = ['B365H', 'B365D', 'B365A']
betting_feats

['B365H', 'B365D', 'B365A']

In [249]:
def compute_meanodds(df, betting_feats):
    """
    """
    home_odds = []
    away_odds = []
    draw_odds = []
    for odd in betting_feats:
        odd_type = odd[-1]
        if odd_type == 'H':
            home_odds.append(odd)
        elif odd_type == 'A':
            away_odds.append(odd)
        else:
            draw_odds.append(odd)
    avg_home_odds = df[home_odds].mean(axis=1)
    avg_away_odds = df[away_odds].mean(axis=1)
    avg_draw_odds = df[draw_odds].mean(axis=1)
    
    ordered_cols = ['HomeOdds', 'AwayOdds', 'DrawOdds'] + df.columns.tolist()
    
    df['HomeOdds'] = avg_home_odds
    df['AwayOdds'] = avg_away_odds
    df['DrawOdds'] = avg_draw_odds
    
    return df[ordered_cols]
df_odd = compute_meanodds(df_noNa_div_date_lastMatch_streak_windate, betting_feats)
df_odd.drop(columns = ['WHH', 'WHD', 'WHA', 'HomeOdds', 'AwayOdds', 'DrawOdds', 'FTHG', 'FTAG', 'FTR'], inplace = True)
df_odd

,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,balance_val,Div 0,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
0,1.727,3.100,4.500,-0.75,2.050,1.850,1702.604858,1611.196045,-1,0,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
1,2.500,2.875,2.625,0.00,1.925,1.975,1685.016113,1665.625732,0,0,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
2,1.571,3.500,5.000,-0.75,1.800,2.100,1718.566284,1649.805298,-1,1,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
3,1.833,3.200,3.750,-0.75,2.025,1.875,1719.916748,1692.120972,-1,1,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
4,1.571,3.250,5.500,-0.75,1.900,2.000,1697.354004,1539.958130,-1,0,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37308,1.670,4.000,5.000,-0.75,1.890,2.010,1856.466064,1735.033813,-1,0,...,2025,0.500000,0.866025,0.417194,0.908818,1,0,0,0,0
37309,2.150,3.100,3.800,-0.25,1.850,2.050,1866.017822,1870.678345,-1,0,...,2025,0.500000,0.866025,0.417194,0.908818,1,0,1,0,1
37310,1.950,4.100,3.500,-0.50,2.010,1.890,1935.029175,1861.455933,-1,0,...,2025,0.500000,0.866025,0.417194,0.908818,0,1,0,1,0
37311,10.000,6.000,1.270,1.75,1.900,2.000,1545.639893,1949.952026,2,0,...,2025,0.500000,0.866025,0.417194,0.908818,0,1,0,1,0


In [250]:
from sklearn.preprocessing import MinMaxScaler
minmax_scaler = MinMaxScaler()
df_odd.loc[:, ['Year']] = minmax_scaler.fit_transform(df_odd.loc[:, ['Year']])

from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
# to_scale = ['HomeWinStreak','AwayWinStreak','HomeWinsToDate', 'AwayWinsToDate', 'HomeTeamELO', 'AwayTeamELO', 'HomeOdds', 'AwayOdds', 'DrawOdds'] + betting_feats
to_scale = ['HomeTeamELO', 'AwayTeamELO'] + betting_feats

df_odd.loc[:, to_scale] = std_scaler.fit_transform(df_odd.loc[:, to_scale])
df_odd

,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,balance_val,Div 0,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
0,-0.509833,-0.624352,-0.016735,-0.75,2.050,1.850,0.098029,-0.701417,-1,0,...,0.0,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
1,-0.074822,-0.813104,-0.533159,0.00,1.925,1.975,-0.055807,-0.225928,0,0,...,0.0,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
2,-0.597623,-0.288795,0.120978,-0.75,1.800,2.100,0.237631,-0.364133,-1,1,...,0.0,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
3,-0.450181,-0.540463,-0.223305,-0.75,2.025,1.875,0.249443,0.005530,-1,1,...,0.0,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
4,-0.597623,-0.498518,0.258691,-0.75,1.900,2.000,0.052103,-1.323739,-1,0,...,0.0,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37308,-0.541911,0.130652,0.120978,-0.75,1.890,2.010,1.443740,0.380410,-1,0,...,1.0,0.500000,0.866025,0.417194,0.908818,1,0,0,0,0
37309,-0.271787,-0.624352,-0.209533,-0.25,1.850,2.050,1.527282,1.565378,-1,0,...,1.0,0.500000,0.866025,0.417194,0.908818,1,0,1,0,1
37310,-0.384339,0.214541,-0.292161,-0.50,2.010,1.890,2.130874,1.484812,-1,0,...,1.0,0.500000,0.866025,0.417194,0.908818,0,1,0,1,0
37311,4.145856,1.808440,-0.906361,1.75,1.900,2.000,-1.274829,2.257899,2,0,...,1.0,0.500000,0.866025,0.417194,0.908818,0,1,0,1,0


In [251]:
# 提取预测数据
testing_row_number = df_noNa_new.shape[0]
df_testing_data = df_odd.iloc[-testing_row_number:].loc[:, df_odd.columns != 'Result']
df_testing_data

,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,balance_val,Div 0,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
37298,0.881865,-0.204906,-0.746614,0.50,2.02,1.88,-0.716472,1.878474,1,0,...,1.0,0.5,0.866025,0.417194,0.908818,3,1,0,1,0
37299,-0.440614,-0.204906,-0.154448,-0.50,1.91,1.99,-0.777909,-1.203591,-1,1,...,1.0,0.5,0.866025,0.417194,0.908818,1,0,1,0,1
37300,-0.570048,0.407487,0.120978,-1.00,2.05,1.85,0.637933,-0.409590,-1,1,...,1.0,0.5,0.866025,0.417194,0.908818,0,0,0,0,0
37301,2.738964,1.598716,-0.889836,1.50,2.00,1.90,-0.352882,1.712089,2,1,...,1.0,0.5,0.866025,0.417194,0.908818,2,0,0,0,0
37302,0.544211,-0.037127,-0.730088,0.50,1.95,1.95,0.483776,1.853672,1,1,...,1.0,0.5,0.866025,0.417194,0.908818,0,1,0,1,0
37303,2.176206,0.550099,-0.856784,1.25,1.83,2.07,-0.240665,2.536777,1,0,...,1.0,0.5,0.866025,0.417194,0.908818,0,0,0,0,1
37304,-0.857055,6.841803,2.875238,-2.50,1.87,2.03,2.731042,-0.832613,-3,0,...,1.0,0.5,0.866025,0.417194,0.908818,3,0,1,0,1
37305,2.457585,1.179269,-0.870556,1.25,2.03,1.87,-1.237088,1.235958,1,0,...,1.0,0.5,0.866025,0.417194,0.908818,0,0,0,0,0
37306,-0.541911,-0.079071,0.258691,-0.75,1.87,2.03,0.757714,0.118782,-1,0,...,1.0,0.5,0.866025,0.417194,0.908818,0,1,0,1,0
37307,-0.328063,-0.121016,-0.319704,-0.25,1.83,2.07,0.955382,0.812623,-1,0,...,1.0,0.5,0.866025,0.417194,0.908818,0,0,1,0,1


In [256]:
### 加载模型
# model_name = 'sklearn_mlp_' + local_time + '.joblib'
# model_name = './' + 'sklearn_svm_2025_01_27_16_48_56.joblib'
# model_name = './' + 'sklearn_svm_randsearch_2025_01_27_17_09_20.joblib'
model_name = './' + 'sklearn_mlp_2025_01_27_17_35_08.joblib'
mlp = load(model_name)
predict_val = mlp.predict(df_testing_data)
series_pre = pd.Series(predict_val, name='Predicted')
series_pre
# compare_result = pd.concat([series_pre, yTe.reset_index()], axis=1)
# compare_result

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- pad1
- pad2
- pad3
